In [37]:
# Importamos la libreria de Python 'fastf1'
import fastf1 as f1

# Cargamos el cache para guardar los datos descargados
f1.Cache.enable_cache('cache')

In [52]:
# Cargamos los datos del Gran Prix Bharain 2024
sesion = f1.get_session(2024, 1, 'Q')

# Mostramos sesion, nombre y fecha de la carrera
print(sesion.name)
print(sesion.event['EventName'])
print(sesion.event['EventDate'])

Qualifying
Bahrain Grand Prix
2024-03-02 00:00:00


In [53]:
sesion.load()

core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.4.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 

In [54]:
sesion.results

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,...,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points
1,1,M VERSTAPPEN,VER,,Red Bull Racing,3671c6,,Max,Verstappen,Max Verstappen,...,NED,1.0,,NaN,0 days 00:01:30.031000,0 days 00:01:29.374000,0 days 00:01:29.179000,NaT,,NaN
16,16,C LECLERC,LEC,,Ferrari,e8002d,,Charles,Leclerc,Charles Leclerc,...,MON,2.0,,NaN,0 days 00:01:30.243000,0 days 00:01:29.165000,0 days 00:01:29.407000,NaT,,NaN
63,63,G RUSSELL,RUS,,Mercedes,27f4d2,,George,Russell,George Russell,...,GBR,3.0,,NaN,0 days 00:01:30.350000,0 days 00:01:29.922000,0 days 00:01:29.485000,NaT,,NaN
55,55,C SAINZ,SAI,,Ferrari,e8002d,,Carlos,Sainz,Carlos Sainz,...,ESP,4.0,,NaN,0 days 00:01:29.909000,0 days 00:01:29.573000,0 days 00:01:29.507000,NaT,,NaN
11,11,S PEREZ,PER,,Red Bull Racing,3671c6,,Sergio,Perez,Sergio Perez,...,MEX,5.0,,NaN,0 days 00:01:30.221000,0 days 00:01:29.932000,0 days 00:01:29.537000,NaT,,NaN
14,14,F ALONSO,ALO,,Aston Martin,229971,,Fernando,Alonso,Fernando Alonso,...,ESP,6.0,,NaN,0 days 00:01:30.179000,0 days 00:01:29.801000,0 days 00:01:29.542000,NaT,,NaN
4,4,L NORRIS,NOR,,McLaren,ff8000,,Lando,Norris,Lando Norris,...,GBR,7.0,,NaN,0 days 00:01:30.143000,0 days 00:01:29.941000,0 days 00:01:29.614000,NaT,,NaN
81,81,O PIASTRI,PIA,,McLaren,ff8000,,Oscar,Piastri,Oscar Piastri,...,AUS,8.0,,NaN,0 days 00:01:30.531000,0 days 00:01:30.122000,0 days 00:01:29.683000,NaT,,NaN
44,44,L HAMILTON,HAM,,Mercedes,27f4d2,,Lewis,Hamilton,Lewis Hamilton,...,GBR,9.0,,NaN,0 days 00:01:30.451000,0 days 00:01:29.718000,0 days 00:01:29.710000,NaT,,NaN
27,27,N HULKENBERG,HUL,,Haas F1 Team,b6babd,,Nico,Hulkenberg,Nico Hulkenberg,...,GER,10.0,,NaN,0 days 00:01:30.566000,0 days 00:01:29.851000,0 days 00:01:30.502000,NaT,,NaN
